In [116]:
import serial
import speech_recognition as sr

In [117]:
import serial.tools.list_ports
comlist = serial.tools.list_ports.comports()

for entry in comlist:
    print("{} : {}".format(entry.device, entry.description))

/dev/cu.SOC : n/a
/dev/cu.MALS : n/a
/dev/cu.usbserial-AC01L4R4 : FT232R USB UART
/dev/cu.Bluetooth-Incoming-Port : n/a


In [118]:
device = '/dev/cu.usbserial'
device = '/dev/cu.usbserial-AC01L4R4'
print("Device to use: {}".format(device))

Device to use: /dev/cu.usbserial-AC01L4R4


In [162]:
def send_command(command, data):
    print("command: {}".format(command))
    print("data   : {}".format(data))
    
    stx = bytearray.fromhex('02')
    etx = bytearray.fromhex('03')
    colon = bytearray.fromhex('3A')
    semicolon = bytearray.fromhex('3B')
    
    s =  stx + bytearray(command.encode('utf-8')) + etx
    
    
    with serial.Serial(
            device,
            baudrate=9600,
            bytesize=serial.SEVENBITS,
            stopbits=serial.STOPBITS_ONE,
            timeout=1,
            parity=serial.PARITY_ODD) as ser:
        num_bytes_send = ser.write(s)
        print("sending {} done".format(num_bytes_send))
        retval = ser.readline()
        print(retval)

In [169]:
cmd_stop = 'OSP'
cmd_play = 'OPL'
cmds = (cmd_stop, cmd_play)
send_command('QOS','')

command: QOS
data   : 
sending 5 done
b'\x02OPS0000E80000\x03'


In [113]:
def identify_command_and_data(text):
    cmd_stop = b'OSP'
    cmd_play = b'OPL'
    cmds = (cmd_stop, cmd_play)
    return cmd_stop,''

In [114]:
def voice_to_text():
    #obtain audio from the microphone
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Say something!")
        audio = r.listen(source)

    print("end of listening")    
    
    # recognize speech using Google Speech Recognition
    
    text = ''
    
    try:
        # for testing purposes, we're just using the default API key
        # to use another API key, use `r.recognize_google(audio, key="GOOGLE_SPEECH_RECOGNITION_API_KEY")`
        # instead of `r.recognize_google(audio)`
        text = r.recognize_google(audio)
        print("Google Speech Recognition thinks you said: " + text)
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service; {0}".format(e))
        
    return text

In [115]:
# Main loop
text = ''
while(text!='end'):
    text = voice_to_text()
    cmd,data = identify_command_and_data(text)
    send_command(cmd, data)

Say something!
end of listening
Google Speech Recognition could not understand audio
controlling vtr
command: b'OSP'
data   : 


AttributeError: 'bytes' object has no attribute 'encode'